Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [17]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [18]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

name    lat    lon
0   Ancona  43.60  13.50
1   Andria  41.23  16.29
2     Bari  41.12  16.87
3  Bergamo  45.70   9.67
4  Bologna  44.50  11.34

In [3]:
DIST_MATRIX

array([[  0.        , 349.30401144, 391.04186819, ..., 223.61470849,
        285.67481712, 266.79616501],
       [349.30401144,   0.        ,  50.17877215, ..., 566.29615596,
        634.93310475, 614.95900451],
       [391.04186819,  50.17877215,   0.        , ..., 604.01577185,
        676.49916849, 654.72206469],
       ...,
       [223.61470849, 566.29615596, 604.01577185, ...,   0.        ,
        104.8569216 ,  63.17924401],
       [285.67481712, 634.93310475, 676.49916849, ..., 104.8569216 ,
          0.        ,  44.69517255],
       [266.79616501, 614.95900451, 654.72206469, ...,  63.17924401,
         44.69517255,   0.        ]])

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [19]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [20]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 1
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

ic| dist: array([[  0.        ,          inf, 391.04186819, ..., 223.61470849,
                  285.67481712, 266.79616501],
                 [349.30401144,          inf,  50.17877215, ..., 566.29615596,
                  634.93310475, 614.95900451],
                 [391.04186819,          inf,   0.        , ..., 604.01577185,
                  676.49916849, 654.72206469],
                 ...,
                 [223.61470849,          inf, 604.01577185, ...,   0.        ,
                  104.8569216 ,  63.17924401],
                 [285.67481712,          inf, 676.49916849, ..., 104.8569216 ,
                    0.        ,  44.69517255],
                 [266.79616501,          inf, 654.72206469, ...,  63.17924401,
                   44.69517255,   0.        ]])
DEBUG:root:step: Andria -> Bari (50.18km)
ic| dist: array([[  0.        ,          inf,          inf, ..., 223.61470849,
                  285.67481712, 266.79616501],
                 [349.30401144,          inf,        

## Second Greedy Algorithm

In [21]:
def has_cycle(vertexs, edges):
    

SyntaxError: incomplete input (3132774802.py, line 2)

In [ ]:
from itertools import accumulate

In [32]:
edges = {frozenset([1, 2]), frozenset([2, 3]), frozenset([3, 4])}
v0, _ = next(iter(edges))
bfs = [v0]
visited = set()

while bfs:
    v = bfs.pop(0)
    visited.add(v)
    adje = {e for e in edges if v0 in e}
    edges -= adje
    ic(adje, edges)
    adjv = frozenset.union(*list(adje)) - {v}
    ic(any(v in visited for v in adjv))
    bfs.extend(list(adjv))
    ic(bfs)


ic| adje: {frozenset({3, 4}), frozenset({2, 3})}
    edges: {frozenset({1, 2})}
ic| any(v in visited for v in adjv): False
ic| bfs: [2, 4]
ic| adje: set(), edges: {frozenset({1, 2})}


TypeError: unbound method frozenset.union() needs an argument

In [25]:
p = [1, 2, 3]
p.pop(0), p

(1, [2, 3])

In [ ]:
segments = [
    (frozenset([c1, c2]), float(DIST_MATRIX[c1, c2]))
    for c1, c2 in combinations(range(len(CITIES)), 2)
]
visited_cities = set()
edges = set()

while segments:
    shortest = next(_ for _ in sorted(segments, key=lambda e: e[1]))
    edges |= {tuple(shortest[0])}
    visited_cities |= shortest[0]
    segments = [
        s for s in segments if not tuple(s[0]) in edges and not has_cycle(edges | {tuple(s[0])})
    ]

In [78]:
[e for e in edges if e[0] == 9 or e[1] == 9]

[(9, 4)]

In [72]:
edges |= {(e2, e1) for e1, e2 in edges}
city = 0
tsp = [city]
while True:
    next_city = next(e2 for e1, e2 in edges if e1 == city)
    tsp.append(next_city)
    edges.remove((city, next_city))
    edges.remove((next_city, city))
    city = next_city

StopIteration: 

In [74]:
tsp

[0, 33, 12, 4, 9]